# California Air Resources Board (CARB)

In [2]:
import requests
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup

### Parameters

In [3]:
URL = "https://ww2.arb.ca.gov/mrr-data"

headers = {
    "Authority": "ww2.arb.ca.gov",
    "Method": "GET",
    "Path": "/mrr-data",
    "Scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "If-None-Match": "1689111602",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}


### Tests

In [4]:
r = requests.get(URL, headers=headers)

In [5]:
soup = BeautifulSoup(r.content, "html.parser")
soup
# Get all hrefs and their text
href_dicts = []
hrefs = soup.find_all("a")
for href in hrefs:
    href_dict = {}
    text=  href.text
    url = href.get("href")

    if "xls" in url:     
        href_dict["name"] = text
        href_dict["url"] = f"https:{url}" 

        href_dicts.append(href_dict)
    

In [6]:
urls_df = pd.DataFrame.from_dict(href_dicts)

### Access one of the files

In [14]:
xlsx_urls = urls_df["url"].tolist()

In [15]:
mega_df = pd.DataFrame([])

for xlsx_url in xlsx_urls:

    response = requests.get(xlsx_url)
    xls_file = pd.ExcelFile(BytesIO(response.content))
    # Create a dictionary of DataFrames, with sheet name as key
    dataframes = {sheet_name: xls_file.parse(sheet_name) for sheet_name in xls_file.sheet_names}

    for key in dataframes.keys():
        if 'GHG Data' in key:
            key_of_interest = key

    ghg_data = dataframes[key_of_interest]
    # Remove rows with several NaNs
    ghg_data = ghg_data.dropna(thresh=10)
    # First row is the header
    ghg_data.columns = ghg_data.iloc[0]
    # Remove rows
    longbeach_df = ghg_data.loc[ghg_data.loc[:, 'City'] == 'Long Beach'].reset_index(drop=True)
    # Remove columns with NaNs
    longbeach_df = longbeach_df.dropna(axis=1, how='all')
    # Remove \n in column names
    longbeach_df.columns = longbeach_df.columns.str.replace('\n', ' ')
    
    mega_df = pd.concat([mega_df, longbeach_df], ignore_index=True)

NameError: name 'xlsx_urls' is not defined